<a href="https://colab.research.google.com/github/ashutoshsoni891/12-Month-Forecast-With-LSTM/blob/master/customNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# format
# https://github.com/openvenues/libpostal
''' 
    pattern  = [('sentence' , {'entities' : [(start , end , entity_name)]} ),
    ]

[('The F15 aircraft uses a lot of fuel', {'entities': [(4, 7, 'aircraft')]}),
 ('did you see the F16 landing?', {'entities': [(16, 19, 'aircraft')]}),
 ('how many missiles can a F35 carry', {'entities': [(24, 27, 'aircraft')]}),
 ('is the F15 outdated', {'entities': [(7, 10, 'aircraft')]}),
 ('does the US still train pilots to dog fight?',
  {'entities': [(0, 0, 'aircraft')]}),
 ('how long does it take to train a F16 pilot',
  {'entities': [(33, 36, 'aircraft')]}),
 ('how much does a F35 cost', {'entities': [(16, 19, 'aircraft')]}),
 ('would it be possible to steal a F15', {'entities': [(32, 35, 'aircraft')]}),
 ('who manufactures the F16', {'entities': 21, 24, 'aircraft')]}),
 ('how many countries have bought the F35',[(
  {'entities': [(35, 38, 'aircraft')]}),
 ('is the F35 a waste of money', {'entities': [(7, 10, 'aircraft')]})]

 '''

 #data source : https://batch.openaddresses.io/data

" \n    pattern  = [('sentence' , {'entities' : [(start , end , entity_name)]} ),\n    ]\n\n[('The F15 aircraft uses a lot of fuel', {'entities': [(4, 7, 'aircraft')]}),\n ('did you see the F16 landing?', {'entities': [(16, 19, 'aircraft')]}),\n ('how many missiles can a F35 carry', {'entities': [(24, 27, 'aircraft')]}),\n ('is the F15 outdated', {'entities': [(7, 10, 'aircraft')]}),\n ('does the US still train pilots to dog fight?',\n  {'entities': [(0, 0, 'aircraft')]}),\n ('how long does it take to train a F16 pilot',\n  {'entities': [(33, 36, 'aircraft')]}),\n ('how much does a F35 cost', {'entities': [(16, 19, 'aircraft')]}),\n ('would it be possible to steal a F15', {'entities': [(32, 35, 'aircraft')]}),\n ('who manufactures the F16', {'entities': [(21, 24, 'aircraft')]}),\n ('how many countries have bought the F35',\n  {'entities': [(35, 38, 'aircraft')]}),\n ('is the F35 a waste of money', {'entities': [(7, 10, 'aircraft')]})]\n\n "

In [2]:
!pip install -U spacy==3.0

     |████████████████████████████████| 12.7MB 233kB/s 
     |████████████████████████████████| 460kB 32.7MB/s 
     |████████████████████████████████| 624kB 32.8MB/s 
     |████████████████████████████████| 9.1MB 186kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [1]:
import spacy
from spacy import displacy
spacy.__version__

'3.0.0'

In [ ]:
# !!python -m spacy download en_core_web_sm
# !!python -m spacy download en_core_web_md
# !!python -m spacy download en_core_web_lg

In [ ]:
# nlp_sm = spacy.load('en_core_web_sm')
# nlp_md = spacy.load('en_core_web_md')
# nlp_lg = spacy.load('en_core_web_lg')



In [ ]:
# string = 'I live in 47A Somani Nagar  City Indore  Pin 452009 which comes under state of Madhya Pradesh. My state name is Madhya Pradesh '
# doc_sm = nlp_sm(string)
# doc_md = nlp_md(string)
# doc_lg = nlp_lg(string)



In [ ]:
# displacy.render(doc_sm, style="ent", jupyter=True)


In [ ]:
# displacy.render(doc_md, style="ent", jupyter=True)


In [ ]:
# displacy.render(doc_lg, style="ent", jupyter=True)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas 
# !wget https://v2.openaddresses.io/batch-prod/collection-us-northeast.zip
# !unzip collection-us-northeast.zip 

     |████████████████████████████████| 1.0MB 7.7MB/s 
     |████████████████████████████████| 15.4MB 218kB/s 
     |████████████████████████████████| 6.6MB 20.8MB/s 
--2021-06-29 05:28:37--  https://v2.openaddresses.io/batch-prod/collection-us-northeast.zip
Resolving v2.openaddresses.io (v2.openaddresses.io)... 89.187.185.162
Connecting to v2.openaddresses.io (v2.openaddresses.io)|89.187.185.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205303407 (196M) [application/zip]
Saving to: ‘collection-us-northeast.zip’

collection-us-north 100%[===================>] 195.79M   497KB/s    in 5m 5s   

2021-06-29 05:33:42 (658 KB/s) - ‘collection-us-northeast.zip’ saved [205303407/205303407]

Archive:  collection-us-northeast.zip
  inflating: us/ri/city_of_cranston-addresses-city.geojson  
  inflating: us/ri/providence-addresses-city.geojson  
  inflating: us/ri/statewide-addresses-state.geojson  
  inflating: us/ct/city_of_bridgeport-addresses-city.geojson  
  in

In [3]:
# import geopandas as gpd
import numpy as np
import pandas as pd
import os
import json
import spacy 
import pickle


In [ ]:

# base_dir = '/content/us'

# for idx , dir in enumerate(os.listdir(base_dir)):
#     for f in os.listdir(os.path.join(base_dir , dir)):

#         fname = os.path.join(base_dir , dir, f)
#         print(f'Reading file..... {fname}')
#         df = gpd.read_file(fname)
#         df.to_csv('master.csv' , mode = 'a', index = None , columns = None)
    

            

Reading file..... /content/us/vt/city_of_burlington-parcels-city.geojson
Reading file..... /content/us/vt/statewide-addresses-state.geojson
Reading file..... /content/us/vt/city_of_burlington-addresses-city.geojson
Reading file..... /content/us/nh/jaffrey-parcels-city.geojson
Reading file..... /content/us/nh/city_of_nashua-addresses-city.geojson
Reading file..... /content/us/nh/jaffrey-addresses-city.geojson
Reading file..... /content/us/nh/city_of_hampstead-addresses-city.geojson
Reading file..... /content/us/nh/milford-addresses-city.geojson
Reading file..... /content/us/nh/statewide-addresses-state.geojson
Reading file..... /content/us/nh/jaffrey-buildings-city.geojson
Reading file..... /content/us/nh/city_of_auburn-addresses-city.geojson
Reading file..... /content/us/ri/city_of_cranston-addresses-city.geojson
Reading file..... /content/us/ri/providence-addresses-city.geojson
Reading file..... /content/us/ri/statewide-addresses-state.geojson
Reading file..... /content/us/me/statewid

In [ ]:

# master_df = pd.read_csv('/content/master.csv' , error_bad_lines=False)
# master_df.drop(['hash' , 'id' , 'geometry'] , axis = 1 , inplace = True)
# master_df = master_df[(master_df['number'] != 'number')]
# master_df[:5]

In [ ]:
# master_df[:50]

,number,street,unit,city,district,region,postcode
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# number street unit city district region postcode 100
# number street unit city district regin 100
# number street unit city district 100
# number street unit city 100
# number street unit 100
# number street 100





In [ ]:
# # %%timeit
# pattern1 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (~master_df['unit'].isna()) & (~master_df['city'].isna()) & (~master_df['district'].isna()) & (~master_df['region'].isna()) & (~master_df['postcode'].isna())]
# pattern2 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (~master_df['unit'].isna()) & (~master_df['city'].isna()) & (~master_df['district'].isna()) & (~master_df['region'].isna()) & (master_df['postcode'].isna())]
# pattern3 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (~master_df['unit'].isna()) & (~master_df['city'].isna()) & (~master_df['district'].isna()) & (master_df['region'].isna()) & (master_df['postcode'].isna())]
# pattern4 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (~master_df['unit'].isna()) & (~master_df['city'].isna()) & (master_df['district'].isna()) & (master_df['region'].isna()) & (master_df['postcode'].isna()) ]
# pattern5 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (~master_df['unit'].isna()) & (master_df['city'].isna()) & (master_df['district'].isna()) & (master_df['region'].isna()) & (master_df['postcode'].isna()) ]
# pattern6 = master_df[(~ master_df['number'].isna()) & (~master_df['street'].isna()) & (master_df['unit'].isna()) & (master_df['city'].isna()) & (master_df['district'].isna()) & (master_df['region'].isna()) & (master_df['postcode'].isna())]


In [ ]:
# master_df.shape  , pattern1.shape , pattern2.shape , pattern3.shape , pattern4.shape , pattern5.shape , pattern6.shape

((7824290, 7),
 (1387136, 7),
 (1922, 7),
 (661, 7),
 (16853, 7),
 (8502, 7),
 (175070, 7))

In [ ]:
# pattern1['address'] = pattern1['number'].astype('str') + ' ' + pattern1['street'] + ' ' + pattern1['unit'] + ' ' + pattern1['city'] + ' ' + pattern1['district'] + ' ' + pattern1['region'] + ' ' + pattern1['postcode'].astype('str')
# pattern2['address'] = pattern2['number'].astype('str') + ' ' + pattern2['street'] + ' ' + pattern2['unit'] + ' ' + pattern2['city'] + ' ' + pattern2['district'] + ' ' + pattern2['region']
# pattern3['address'] = pattern3['number'].astype('str') + ' ' + pattern3['street'] + ' ' + pattern3['unit'] + ' ' + pattern3['city'] + ' ' + pattern3['district'] 
# pattern4['address'] = pattern4['number'].astype('str') + ' ' + pattern4['street'] + ' ' + pattern4['unit'] + ' ' + pattern4['city'] 
# pattern5['address'] = pattern5['number'].astype('str') + ' ' + pattern5['street'] + ' ' + pattern5['unit']
# pattern6['address'] = pattern6['number'].astype('str') + ' ' + pattern6['street'] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
address = json.load(open('address.json' , 'r'))
address[:5]

[[{'city': 'McCulloughmouth',
   'company': 'Buckridge-Lubowitz',
   'country': 'Austria',
   'credit': {'code': 734,
    'expirationDate': '09/22',
    'name': 'Dr. Ewald Breitenberg',
    'number': '4716571762685003',
    'type': 'Visa'},
   'gender': '-',
   'name': 'Vinnie Metz',
   'phone': '797.935.5274 x72107',
   'postcode': '22142',
   'state': '-',
   'street': '-',
   'streetAddress': '524 Parker Motorway Suite 525'}],
 [{'city': 'West Gilbertoport',
   'company': 'Ziemann, Mohr and Wintheiser',
   'country': 'Austria',
   'credit': {'code': 326,
    'expirationDate': '02/23',
    'name': 'Miss Alexandrea Dach II',
    'number': '2221116828487259',
    'type': 'MasterCard'},
   'gender': '-',
   'name': 'Gerda Ernser',
   'phone': '1-236-628-8073 x8097',
   'postcode': '14938',
   'state': '-',
   'street': '-',
   'streetAddress': '86286 Williamson Tunnel'}],
 [{'city': 'Saint-Hubert',
   'company': 'Georges SPRL',
   'country': 'Belgium',
   'credit': {'code': 466,
    'ex

In [7]:
text_df = pd.read_csv('/content/drive/MyDrive/10ksamplesgdelt.csv')
text_df[:5]

,DATEADDED,EventRootCode,SOURCEURL,title,text
0,20170814084500,1,https://www.businessinsider.com.au/theresa-may...,Theresa May's Brexit plans are allegedly being...,LONDON — Prime Minister Theresa May returns fr...
1,20170814084500,1,http://newsblaze.com/business/legal/a-declarat...,A Declaration That Open Carry Is the Right,9th Circuit Court of Appeals Oral Argument in ...
2,20170814084500,1,http://www.radionz.co.nz/international/pacific...,Confusion over pharmacy strike in New Caledonia,There is confusion in New Caledonia over which...
3,20170814084500,1,http://www.brisbanetimes.com.au/victoria/devel...,Developer and Liberal Party donor makes $36m p...,A developer and Liberal Party donor who gained...
4,20170814084500,1,http://english.anhuinews.com/system/2017/08/14...,China Focus: China Fights for Final Victory ov...,"BEIJING, Aug. 13 (Xinhua) -- In the late 1930s..."


In [8]:
text_df['text'] = text_df['text'].apply(lambda X : X.replace('\n' , '').strip())

In [9]:
text_df[:5]

,DATEADDED,EventRootCode,SOURCEURL,title,text
0,20170814084500,1,https://www.businessinsider.com.au/theresa-may...,Theresa May's Brexit plans are allegedly being...,LONDON — Prime Minister Theresa May returns fr...
1,20170814084500,1,http://newsblaze.com/business/legal/a-declarat...,A Declaration That Open Carry Is the Right,9th Circuit Court of Appeals Oral Argument in ...
2,20170814084500,1,http://www.radionz.co.nz/international/pacific...,Confusion over pharmacy strike in New Caledonia,There is confusion in New Caledonia over which...
3,20170814084500,1,http://www.brisbanetimes.com.au/victoria/devel...,Developer and Liberal Party donor makes $36m p...,A developer and Liberal Party donor who gained...
4,20170814084500,1,http://english.anhuinews.com/system/2017/08/14...,China Focus: China Fights for Final Victory ov...,"BEIJING, Aug. 13 (Xinhua) -- In the late 1930s..."


In [ ]:
#convert text to Setences ( Sentecizer ) -> find GPE -> take 3-4 sentences -> replace GPE with address of patterns (1-6)

#try spacy lg model

In [10]:
!python -m spacy download en_core_web_sm


2021-07-09 08:19:20.941824: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6MB 224kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('sentencizer')

In [12]:
import sys


sentence = []
start_char = []
end_char = []
entity = []


for doc in nlp.pipe(text_df['text'][2000:] , disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):

    for sent in doc.sents:

        print(sent.ents)

        if sent.text in sentence:
            break

        for ent in sent.ents:


            if ent.label_ == 'GPE':
                
                sentence.append(sent.text)
                start_char.append(ent.start_char - ent.sent.start_char)
                end_char.append(ent.end_char-ent.sent.start_char)
                entity.append(ent.text)
                

                if len(entity) == 6710: #600 X 6 patterns + 3110 new addresses
                    print('EXIT-ING the programme')
                    sys.exit()
                    
                break
 




Streaming output truncated to the last 5000 lines.
[nearly ever year, the dry season, 1997]
[tens of thousands]
[Singapore, 1997, Robert Field, Columbia University, NASA, Goddard Institute for Space Studies, US]
[2015]
[Indonesia, this year, Jakarta, more than 20,000]
[El Nino, Indonesia, a month]
[Malaysia, Singapore]
[]
[Herry Purnomo, Indonesia, Centre for International Forestry Research, AFP, the dry season, next month]
[Singapore, Friday]
[]
[overnight]
[Malaysia, weeks, Malaysia]
[Lim Teck Wyn, Kuala Lumpur]
[annual]
[]
[NASA, Global Fire Emissions Database, 600 million tonnes, year, Germany, output.-AFPFollow Coconuts, Instagram, Twitter]
[England, Wales]
[under 18, 75]
[]
[first]
[the first three months, the National Police Chiefs' Council]
[]
[]
[Penny Woods, the British Lung Foundation]
[More than 430,000, second, each week]
[UK, U.S., California, Utah, Maine]
[UK]
[]
[Simon Clark]
[Steve White, the Police Federation]
[]
[]
[Gina Radford, England]
[]
[Scottish, Parliament]
[2

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
len(sentence) , len(entity) , len(start_char) , len(end_char)

(6710, 6710, 6710, 6710)

In [14]:
for sen , ent , start , end in zip(sentence , entity , start_char , end_char ):
    print(ent, start , end , sen  , sep = ('**************'))

Streaming output truncated to the last 5000 lines.
Barcelona**************188**************197**************"Salvamento Maritimo, the Spanish search and rescue team, said the bodies of the women, believed to be aged 33 and 36, had been taken to the neighbouring town of Blanes, 40 miles north of Barcelona.
London**************139**************145**************Zac Goldsmith is opposed to the prospect of a third runway at HeathrowZac Goldsmith has been confirmed today as Conservative candidate for London mayor in succession to Boris Johnson in a development that places Heathrow expansion plans in fresh doubt.
Richmond Park**************11**************24**************The MP for Richmond Park in southwest London, who is a prominent environmental campaigner, is vehemently opposed to the prospect of a third runway at Heathrow and has pledged to resign his seat should the government approve it.
UK**************58**************60**************AboutManufacturing Management (MM) celebrates the b

In [15]:
data_df = pd.DataFrame({'sentence' : sentence , 'entity_start_idx' : start_char , 'entity_end_idx' : end_char , 'entity' :entity})
data_df[:5]

,sentence,entity_start_idx,entity_end_idx,entity
0,"Aihara, clearly humbled by the honor, said she...",0,6,Aihara
1,"”For Aihara, nursing has been a way of life fo...",5,11,Aihara
2,I think I subconsciously decided to take the s...,65,71,Aihara
3,Aihara was more than interested.,0,6,Aihara
4,"“It was the best year of my life, I mean, I wo...",68,77,’s County


In [16]:
data_df.shape

(6710, 4)

In [ ]:
# data_df['address'] = pd.concat([pattern1['address'][1:601] , pattern2['address'][:600] , pattern3['address'][:600], pattern4['address'][:600] , pattern5['address'][:600] ,pattern6['address'][:600]] , axis = 0).tolist()

NameError: ignored

In [ ]:
# data_df.isna().sum()

sentence            0
entity_start_idx    0
entity_end_idx      0
entity              0
address             0
dtype: int64

In [17]:
temp = pd.read_csv('/content/drive/MyDrive/spacycustomner/pseudoaddressforspacyner.csv')
temp['new_entity_end_idx'] = 0
temp['sentence'] = temp['sentence'].apply(lambda X : X.replace('\n' , '').strip())
temp['address'] = temp['address'].apply(lambda X : X.title())
print(temp.shape)
temp[:5]


(3600, 6)


,sentence,entity_start_idx,entity_end_idx,entity,address,new_entity_end_idx
0,"At the Ryuwon shoe factory in Pyongyang, Adida...",30,39,Pyongyang,600 Four Rod Rd #9 Berlin Hartford Connecticut...,0
1,“North Korean people condemn Japan in politics...,31,36,Japan,600 Four Rod Rd #3 Berlin Hartford Connecticut...,0
2,Photograph: Ekaterina Ochagavia/The GuardianAt...,87,96,Pyongyang,600 Four Rod Rd #2 Berlin Hartford Connecticut...,0
3,Photograph: Ekaterina Ochagavia/The GuardianCo...,82,93,North Korea,600 Four Rod Rd #1 Berlin Hartford Connecticut...,0
4,“I think he (Kim) really wants this lifestyle;...,221,226,Seoul,600 Four Rod Rd #7 Berlin Hartford Connecticut...,0


In [18]:
data_df['address'] = 0
data_df['new_entity_end_idx'] = 0
data_df['address'][:3600] = temp['address']
data_df['address'][3600:] = [a[0]['streetAddress'] for a in address]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
data_df[:5]

,sentence,entity_start_idx,entity_end_idx,entity,address,new_entity_end_idx
0,"Aihara, clearly humbled by the honor, said she...",0,6,Aihara,600 Four Rod Rd #9 Berlin Hartford Connecticut...,0
1,"”For Aihara, nursing has been a way of life fo...",5,11,Aihara,600 Four Rod Rd #3 Berlin Hartford Connecticut...,0
2,I think I subconsciously decided to take the s...,65,71,Aihara,600 Four Rod Rd #2 Berlin Hartford Connecticut...,0
3,Aihara was more than interested.,0,6,Aihara,600 Four Rod Rd #1 Berlin Hartford Connecticut...,0
4,"“It was the best year of my life, I mean, I wo...",68,77,’s County,600 Four Rod Rd #7 Berlin Hartford Connecticut...,0


In [ ]:
# temp = pd.concat([data_df , temp[['address','new_entity_end_idx']]] , axis = 1)

In [ ]:
# temp[:5]

,sentence,entity_start_idx,entity_end_idx,entity,address,new_entity_end_idx
0,"At the Ryuwon shoe factory in Pyongyang, Adida...",30,39,Pyongyang,600 Four Rod Rd #9 Berlin Hartford Connecticut...,0
1,The factory showroom boasts virtual copies of ...,127,132,Asics,600 Four Rod Rd #3 Berlin Hartford Connecticut...,0
2,Footwear design may seem an incongruous concer...,177,179,US,600 Four Rod Rd #2 Berlin Hartford Connecticut...,0
3,“North Korean people condemn Japan in politics...,29,34,Japan,600 Four Rod Rd #1 Berlin Hartford Connecticut...,0
4,Photograph: Ekaterina Ochagavia/The GuardianAt...,85,94,Pyongyang,600 Four Rod Rd #7 Berlin Hartford Connecticut...,0


In [20]:
temp = data_df

In [ ]:
# temp['address'][:-200] = temp['address'][:-1000].apply(lambda X : X.lower())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
for idx in range(len(temp)):
     
    to_replace = temp['sentence'][idx][temp['entity_start_idx'][idx] : temp['entity_end_idx'][idx]]

    temp['sentence'][idx] = temp['sentence'][idx].replace(to_replace  , temp['address'][idx] ,1 )
    temp['new_entity_end_idx'][idx] =  temp['entity_start_idx'][idx] + len(temp['address'][idx])
    print(to_replace , temp['entity_end_idx'][idx] , temp['new_entity_end_idx'][idx])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Streaming output truncated to the last 5000 lines.
Barcelona 197 225
London 145 176
Richmond Park 24 48
UK 60 95
UK 163 198
Washington 157 184
Washington 113 140
Iran 196 234
Texas 57 89
America 301 331
Richmond 8 37
Kentucky 65 94
Nuremberg 154 187
Birmingham Jail 63 89
the United States 68 92
U.S. 30 67
Houston 60 94
the United States 32 56
Ireland 10 41
Ireland 59 90
Vusala AbbasovaKazakhstan 28 32
Kazenergy 57 77
Russia 26 55
India 24 54
Kazakhstan 10 36
China 30 61
India 86 117
Afghanistan 71 96
India 40 70
Kazakhstan 23 48
India 74 104
KazTransOil 212 236
Kazakhstan 17 42
Kazakhstan 59 84
Kazakhstan 22 48
JERUSALEM 9 36
Eitam 5 35
Israel 55 84
Pray 37 68
P.A. 47 79
New York 135 163
Rochdale 130 158
Memphis 29 58
Mabuhay 57 86
Legarda 7 36
the Russian Federation 126 137
Ukraine 7 29
Eurasia 184 207
Russia 6 30
Kazakhstan 86 106
Kazakhstan 67 87
the Russian Federation 81 89
Azerbaijan 117 137
Moscow 135 159
NEW YORK 8 30
Kansas City 98 117
Detroit 42 65
China 165 190
Thailand 8 30


In [22]:
temp[:5]

,sentence,entity_start_idx,entity_end_idx,entity,address,new_entity_end_idx
0,600 Four Rod Rd #9 Berlin Hartford Connecticut...,0,6,Aihara,600 Four Rod Rd #9 Berlin Hartford Connecticut...,51
1,”For 600 Four Rod Rd #3 Berlin Hartford Connec...,5,11,Aihara,600 Four Rod Rd #3 Berlin Hartford Connecticut...,56
2,I think I subconsciously decided to take the s...,65,71,Aihara,600 Four Rod Rd #2 Berlin Hartford Connecticut...,116
3,600 Four Rod Rd #1 Berlin Hartford Connecticut...,0,6,Aihara,600 Four Rod Rd #1 Berlin Hartford Connecticut...,51
4,"“It was the best year of my life, I mean, I wo...",68,77,’s County,600 Four Rod Rd #7 Berlin Hartford Connecticut...,119


In [23]:
# pattern  = [('sentence' , {'entities' : [(start , end , entity_name)]} ),
# ]


pattern = []
for idx in range(len(temp) ):
  
    expression = [temp['sentence'][idx] , {"entities" : [(int(temp['entity_start_idx'][idx]) , int(temp['new_entity_end_idx'][idx]) , "address" )]} ]
    pattern.append(expression)

In [24]:
pattern[:10]

[['600 Four Rod Rd #9 Berlin Hartford Connecticut 6037, clearly humbled by the honor, said she feels she could share the award with a number of “incredibly fabulous and fantastic nurses who equally deserve such recognition.',
  {'entities': [(0, 51, 'address')]}],
 ['”For 600 Four Rod Rd #3 Berlin Hartford Connecticut 6037, nursing has been a way of life for the past 37 years.',
  {'entities': [(5, 56, 'address')]}],
 ['I think I subconsciously decided to take the same path she did,” 600 Four Rod Rd #2 Berlin Hartford Connecticut 6037 said. “',
  {'entities': [(65, 116, 'address')]}],
 ['600 Four Rod Rd #1 Berlin Hartford Connecticut 6037 was more than interested.',
  {'entities': [(0, 51, 'address')]}],
 ['“It was the best year of my life, I mean, I woke up every day in God600 Four Rod Rd #7 Berlin Hartford Connecticut 6037,” Aihara said.',
  {'entities': [(68, 119, 'address')]}],
 ['In 600 Four Rod Rd #8 Berlin Hartford Connecticut 6037’s case, two patients nominated her for the hono

In [ ]:
pattern[4][0][85:136]

'600 Four Rod Rd #7 Berlin Hartford Connecticut 6037'

In [25]:
json.dump(pattern[:-100] , open('train.json' ,'w'))
json.dump(pattern[-100:] , open('eval.json' ,'w'))


In [ ]:
# !rm -rf /content/model-best /content/model-last

In [ ]:
# !cp -r /content/drive/MyDrive/spacycustomner ./

In [26]:
!git clone https://github.com/Vijayvj1/Custom_NER_Spacy3/
%cd Custom_NER_Spacy3
!pip install -r requirements.txt

Cloning into 'Custom_NER_Spacy3'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 3), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (25/25), done.
/content/Custom_NER_Spacy3
     |████████████████████████████████| 7.8MB 9.8MB/s 
     |████████████████████████████████| 12.8MB 240kB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 4.2MB 51.7MB/s 
     |████████████████████████████████| 174kB 60.1MB/s 
     |████████████████████████████████| 112kB 43.1MB/s 
     |████████████████████████████████| 9.1MB 21.7MB/s 
     |████████████████████████████████| 122kB 52.2MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13476 sha256=80622b870a727784b66e6023c0e32f946335662d7a8830f4ebff781563370260
  Stored in directory: /root/.cache/pip/wheels/92

In [27]:
tempjson = json.load(open('/content/train.json' , 'r') )
tempjson[:5]

[['600 Four Rod Rd #9 Berlin Hartford Connecticut 6037, clearly humbled by the honor, said she feels she could share the award with a number of “incredibly fabulous and fantastic nurses who equally deserve such recognition.',
  {'entities': [[0, 51, 'address']]}],
 ['”For 600 Four Rod Rd #3 Berlin Hartford Connecticut 6037, nursing has been a way of life for the past 37 years.',
  {'entities': [[5, 56, 'address']]}],
 ['I think I subconsciously decided to take the same path she did,” 600 Four Rod Rd #2 Berlin Hartford Connecticut 6037 said. “',
  {'entities': [[65, 116, 'address']]}],
 ['600 Four Rod Rd #1 Berlin Hartford Connecticut 6037 was more than interested.',
  {'entities': [[0, 51, 'address']]}],
 ['“It was the best year of my life, I mean, I woke up every day in God600 Four Rod Rd #7 Berlin Hartford Connecticut 6037,” Aihara said.',
  {'entities': [[68, 119, 'address']]}]]

In [28]:
tempjson[4][0][68:119]

'600 Four Rod Rd #7 Berlin Hartford Connecticut 6037'

In [29]:
!python scripts/convert.py en /content/train.json train.spacy
!python scripts/convert.py en /content/eval.json eval.spacy

2021-07-09 08:24:59.022785: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
scripts/convert.py:22: UserWarning: Skipping entity [68, 119, address] in the following text because the character span '600 Four Rod Rd #7 Berlin Hartford Connecticut 6037' does not align with token boundaries:

'“It was the best year of my life, I mean, I woke up every day in God600 Four Rod Rd #7 Berlin Hartford Connecticut 6037,” Aihara said.'

  warnings.warn(msg)
scripts/convert.py:22: UserWarning: Skipping entity [244, 292, address] in the following text because the character span 'eed that subsidy removal is the best for Nigeria' does not align with token boundaries:

'In his brief, the Executive Director of Conscience 10 Main St #102 Berlin Hartford Connecticut 6037, Comrade Tosin Adeyanju said a coalition of non-governmental organizations led by Conscience Nigeria had deliberated upon the new policy and agreed that subsidy removal i

In [4]:
!pip install -U spacy-transformers transformers

     |████████████████████████████████| 2.5MB 12.8MB/s 
     |████████████████████████████████| 1.0MB 37.9MB/s 
     |████████████████████████████████| 901kB 38.3MB/s 
     |████████████████████████████████| 3.3MB 15.5MB/s 
ERROR: spacy-transformers 1.0.3 has requirement transformers<4.7.0,>=3.4.0, but you'll have transformers 4.8.2 which is incompatible.


In [ ]:
# !https://spacy.io/usage/training

In [31]:
# !python -m spacy init config --lang en --pipeline ner base_config.cfg --force
!python -m spacy init fill-config /content/base_config.cfg /content/Custom_NER_Spacy3/config.cfg

2021-07-09 08:27:14.716676: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
/content/Custom_NER_Spacy3/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [33]:
# !python -m spacy train base_config.cfg --output /content --paths.train train.spacy --paths.dev eval.spacy  --training.max_steps 1000 --gpu-id -0
!python -m spacy train -g 0 /content/Custom_NER_Spacy3/config.cfg --output /content


2021-07-09 08:29:35.437220: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-07-09 08:29:37,856] [INFO] Set up nlp object from config
[2021-07-09 08:29:37,865] [INFO] Pipeline: ['transformer', 'ner']
[2021-07-09 08:29:37,869] [INFO] Created vocabulary
[2021-07-09 08:29:37,869] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

In [ ]:
# !python -m spacy debug config config.cfg


2021-06-30 07:49:38.336983: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [2]:
1+2

3

In [1]:
import spacy
spacy.__version__

'3.0.0'

In [9]:
nlp1 = spacy.load('/content/drive/MyDrive/model-best') #load the best model
string = 'I am wondering if someone has a house near my building whose address is 18D, Omkar Marg, Gandhi Nagar , Indore 453112.'
string2 = 'I live in 47 Somani Nagar  Indore 452005 Madhya Pradesh'
doc = nlp1(string) # input sample tex

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter



/usr/local/lib/python3.7/dist-packages/spacy/util.py:707: UserWarning: [W095] Model 'en_pipeline' (0.0.0) requires spaCy >=3.0.6,<3.1.0 and is incompatible with the current version (3.0.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
!cp -r /content/model-best /content/model-last /content/drive/MyDrive/spacycustomner

In [ ]:
nlp2 = spacy.load(R"/content/model-last") #load the best model
doc = nlp1("I like CA") # input sample tex

displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

OSError: ignored